# Topic Modelling
Data Science Workflow:
1. Start with a question 
2. Get & Clean the data   
3. Perform EDA
4. Apply Techniques
    - **Input:** clean data, plus we have verified that the data makes sense
    - **NLP Techniques:** advanced analysis techniques, in this case, they are specifically designed for text data
    - **Output:** additional insights about our data to help us answer our question, "how is Ali Wong different?"
5. Share Insights

## Topic Modeling
- **Input:** A document-term matrix. Each topic will consist of a set of wprds where order does not matter, so we are going to start with the bag of words format.
- **genism:** genism is a Python toolkit built by Radim Rehurek specifically for topic modeling. We are going to a popular topic modeling technique called Latent Dirichlet Allocation (LDA). We are also going to use **nltk** for some parts-of-speech tagging.
- **Output:** Our goal is to find themes across various comedy routines, and see which comedians tend to talk about which themes.

### Latent Dirichlet Allocation
Latent -- > hidden <br>
Dirichlet --> types of probability distribution

### How LDA works?
- **Goal:** - You want LDA to learn the `topic mix` in each document, and the `word mix` in each topic
- **Input:** - Document-Term Matrix, number of topics, number of iterations
- `genism` will go through the process of finding the best word distribution for each topic and best topic distribution for each document.
- **Output:** *The top words in each topic*. It is your job as a human to interpret this and see if the results makes sense. If not, try altering the parameters - terms in the document-term matrix, number of topics, number of iterations, etc, *Stop when the topics make sense*.
- This is a probabilistic approach to topic modelling. There are also matriz factorization techniques for topic modeling such as **Latent Semantic Indexing (LSI)** and **Non-Negative Matrix Factorization (NMF)**


## 1.2 Topic Modelling -  Attempt #1 (All Text)

In [1]:
# Let's read in our document-term matrix
import pandas as pd
import dill

data = pd.read_pickle('dtm_stop.pkl')
data

,aaaaah,aaaaahhhhhhh,aaaaauuugghhhhhh,aaaahhhhh,aaah,aah,abc,abcs,ability,abject,...,zee,zen,zeppelin,zero,zillion,zombie,zombies,zoning,zoo,éclair
ali,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,1,0,0,0,0,0,0,1,0,0,...,0,0,0,1,1,1,1,1,0,0
bo,0,1,1,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
dave,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,2,1,0,1,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
louis,0,0,0,0,0,3,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0


In [2]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,ali,anthony,bill,bo,dave,hasan,jim,joe,john,louis,mike,ricky
aaaaah,0,0,1,0,0,0,0,0,0,0,0,0
aaaaahhhhhhh,0,0,0,1,0,0,0,0,0,0,0,0
aaaaauuugghhhhhh,0,0,0,1,0,0,0,0,0,0,0,0
aaaahhhhh,0,0,0,1,0,0,0,0,0,0,0,0
aaah,0,0,0,0,1,0,0,0,0,0,0,0


In [4]:
# We're going to put the term-document matrix into a new gensim format, 
# from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [5]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = dill.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())
# id2word

In [6]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.010*"fucking" + 0.007*"fuck" + 0.006*"shit" + 0.006*"theyre" + 0.006*"say" + 0.005*"thing" + 0.005*"good" + 0.005*"went" + 0.005*"hes" + 0.005*"didnt"'),
 (1,
  '0.005*"shit" + 0.005*"want" + 0.005*"going" + 0.005*"say" + 0.004*"cause" + 0.004*"dad" + 0.004*"hey" + 0.004*"did" + 0.004*"hes" + 0.004*"really"')]

In [7]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.011*"fucking" + 0.007*"went" + 0.007*"fuck" + 0.007*"shit" + 0.006*"good" + 0.005*"didnt" + 0.005*"going" + 0.005*"man" + 0.004*"goes" + 0.004*"day"'),
 (1,
  '0.007*"say" + 0.005*"going" + 0.005*"dad" + 0.005*"love" + 0.005*"little" + 0.005*"day" + 0.005*"want" + 0.005*"fucking" + 0.004*"did" + 0.004*"didnt"'),
 (2,
  '0.008*"shit" + 0.008*"fucking" + 0.007*"fuck" + 0.006*"cause" + 0.006*"theyre" + 0.005*"theres" + 0.005*"hes" + 0.005*"life" + 0.005*"really" + 0.005*"thing"')]

In [8]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.009*"shit" + 0.008*"fucking" + 0.007*"fuck" + 0.006*"theyre" + 0.005*"didnt" + 0.005*"cause" + 0.005*"say" + 0.005*"hes" + 0.005*"thing" + 0.005*"man"'),
 (1,
  '0.006*"say" + 0.006*"id" + 0.006*"fucking" + 0.005*"want" + 0.005*"love" + 0.004*"didnt" + 0.004*"little" + 0.004*"says" + 0.004*"going" + 0.004*"hes"'),
 (2,
  '0.001*"dad" + 0.001*"going" + 0.001*"want" + 0.001*"love" + 0.001*"life" + 0.000*"say" + 0.000*"look" + 0.000*"guys" + 0.000*"hes" + 0.000*"school"'),
 (3,
  '0.009*"fucking" + 0.008*"going" + 0.007*"went" + 0.006*"want" + 0.006*"dad" + 0.006*"fuck" + 0.005*"day" + 0.005*"say" + 0.005*"shes" + 0.005*"love"')]

## 1.3 Topic Modeling - Attempt # 2 (Nouns Only)

In [9]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [10]:

# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,transcript
ali,ladies and gentlemen please welcome to the sta...
anthony,thank you thank you thank you san francisco th...
bill,all right thank you thank you very much thank ...
bo,bo what old macdonald had a farm e i e i o and...
dave,this is dave he tells dirty jokes for a living...
hasan,whats up davis whats up im home i had to bring...
jim,ladies and gentlemen please welcome to the sta...
joe,ladies and gentlemen welcome joe rogan what th...
john,all right petunia wish me luck out there you w...
louis,intro fade the music out lets roll hold there ...


In [11]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\yrobi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [12]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yrobi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
ali,ladies gentlemen stage ali hi thank hello na s...
anthony,thank thank people i em i francisco city world...
bill,thank thank pleasure georgia area oasis i june...
bo,macdonald farm e i o farm pig e i i snort macd...
dave,jokes living stare work profound train thought...
hasan,whats davis whats home i netflix la york i son...
jim,ladies gentlemen stage mr jim jefferies thank ...
joe,ladies gentlemen joe fuck thanks phone fuckfac...
john,petunia thats hello hello chicago thank crowd ...
louis,intro music lets lights lights thank i i place...


In [14]:

# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
# Convert frozenset to a list
stop_words_list = list(stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words_list)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,aaaaahhhhhhh,aaaaauuugghhhhhh,aaaahhhhh,aah,abc,abcs,ability,abortion,abortions,abuse,...,yummy,ze,zealand,zee,zeppelin,zillion,zombie,zombies,zoo,éclair
ali,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
anthony,0,0,0,0,0,0,0,2,0,0,...,0,0,10,0,0,0,0,0,0,0
bill,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,1,1,1,0,0
bo,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dave,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
louis,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())


In [16]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.011*"shit" + 0.009*"life" + 0.008*"man" + 0.008*"hes" + 0.008*"fuck" + 0.007*"thing" + 0.007*"cause" + 0.007*"guy" + 0.007*"gon" + 0.006*"way"'),
 (1,
  '0.011*"day" + 0.009*"thing" + 0.007*"joke" + 0.006*"way" + 0.006*"man" + 0.006*"things" + 0.006*"hes" + 0.005*"years" + 0.005*"cause" + 0.005*"life"')]

In [17]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.009*"dad" + 0.007*"hes" + 0.007*"shes" + 0.007*"life" + 0.007*"years" + 0.006*"day" + 0.006*"thing" + 0.005*"joke" + 0.005*"man" + 0.004*"way"'),
 (1,
  '0.011*"day" + 0.009*"thing" + 0.008*"cause" + 0.007*"way" + 0.006*"guy" + 0.005*"things" + 0.005*"man" + 0.005*"night" + 0.005*"women" + 0.005*"house"'),
 (2,
  '0.015*"shit" + 0.010*"man" + 0.010*"life" + 0.010*"fuck" + 0.009*"thing" + 0.009*"hes" + 0.008*"gon" + 0.008*"lot" + 0.008*"guy" + 0.008*"cause"')]

In [18]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.013*"dad" + 0.009*"mom" + 0.008*"shes" + 0.007*"life" + 0.007*"hey" + 0.007*"parents" + 0.007*"day" + 0.007*"way" + 0.006*"house" + 0.006*"clinton"'),
 (1,
  '0.014*"shit" + 0.011*"thing" + 0.011*"man" + 0.010*"fuck" + 0.010*"life" + 0.009*"day" + 0.009*"cause" + 0.009*"guy" + 0.009*"hes" + 0.008*"lot"'),
 (2,
  '0.007*"id" + 0.007*"thing" + 0.007*"day" + 0.007*"hes" + 0.007*"way" + 0.006*"stuff" + 0.006*"point" + 0.006*"things" + 0.006*"years" + 0.006*"joke"'),
 (3,
  '0.014*"joke" + 0.011*"day" + 0.007*"thing" + 0.007*"anthony" + 0.007*"school" + 0.006*"family" + 0.006*"jokes" + 0.006*"grandma" + 0.006*"lot" + 0.006*"baby"')]

In [19]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [20]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
ali,ladies gentlemen welcome stage ali wong hi wel...
anthony,thank san francisco thank good people surprise...
bill,right thank thank pleasure greater atlanta geo...
bo,old macdonald farm e i i o farm pig e i i snor...
dave,dirty jokes living stare most hard work profou...
hasan,whats davis whats im home i netflix special la...
jim,ladies gentlemen welcome stage mr jim jefferie...
joe,ladies gentlemen joe fuck san francisco thanks...
john,right petunia august thats good right hello he...
louis,intro music lets lights lights thank much i i ...


In [21]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words_list, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aaaaah,aaaaahhhhhhh,aaaaauuugghhhhhh,aaaahhhhh,aah,abc,abcs,ability,abject,able,...,ze,zealand,zee,zeppelin,zero,zillion,zombie,zombies,zoo,éclair
ali,0,0,0,0,0,1,0,0,0,2,...,0,0,0,0,0,0,1,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,10,0,0,0,0,0,0,0,0
bill,1,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,1,1,1,0,0
bo,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,1,...,0,0,2,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,1
louis,0,0,0,0,3,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [23]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.006*"joke" + 0.004*"mom" + 0.004*"parents" + 0.003*"hasan" + 0.003*"jenny" + 0.003*"jokes" + 0.002*"anthony" + 0.002*"doctor" + 0.002*"door" + 0.002*"comedy"'),
 (1,
  '0.003*"dick" + 0.003*"wife" + 0.003*"ass" + 0.003*"dog" + 0.003*"son" + 0.003*"mom" + 0.003*"gun" + 0.002*"bo" + 0.002*"gay" + 0.002*"clinton"')]

In [24]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.004*"bo" + 0.004*"parents" + 0.004*"hasan" + 0.004*"jenny" + 0.003*"comedy" + 0.003*"um" + 0.003*"repeat" + 0.003*"class" + 0.003*"guns" + 0.003*"love"'),
 (1,
  '0.005*"ok" + 0.005*"joke" + 0.004*"tit" + 0.004*"nuts" + 0.003*"husband" + 0.003*"older" + 0.003*"dead" + 0.003*"food" + 0.003*"jenner" + 0.003*"ha"'),
 (2,
  '0.005*"mom" + 0.004*"joke" + 0.004*"wife" + 0.003*"mad" + 0.003*"dick" + 0.003*"clinton" + 0.003*"friend" + 0.003*"ahah" + 0.003*"parents" + 0.003*"anthony"')]

In [25]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.009*"joke" + 0.006*"mom" + 0.004*"jokes" + 0.004*"clinton" + 0.003*"parents" + 0.003*"anthony" + 0.003*"cow" + 0.003*"president" + 0.003*"jenner" + 0.003*"wife"'),
 (1,
  '0.006*"hasan" + 0.005*"guns" + 0.005*"mom" + 0.004*"class" + 0.004*"ass" + 0.004*"parents" + 0.004*"brown" + 0.004*"son" + 0.004*"girlfriend" + 0.003*"gun"'),
 (2,
  '0.006*"bo" + 0.005*"repeat" + 0.005*"ahah" + 0.005*"gay" + 0.005*"eye" + 0.004*"young" + 0.004*"contact" + 0.004*"tit" + 0.004*"ok" + 0.004*"nigga"'),
 (3,
  '0.005*"jenny" + 0.003*"husband" + 0.003*"ok" + 0.003*"gun" + 0.003*"accident" + 0.003*"morning" + 0.003*"sense" + 0.003*"sudden" + 0.003*"love" + 0.003*"marriage"')]

In [26]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.005*"jenny" + 0.004*"guns" + 0.004*"ass" + 0.004*"class" + 0.003*"girls" + 0.003*"texas" + 0.003*"girlfriend" + 0.003*"friend" + 0.003*"door" + 0.003*"morning"'),
 (1,
  '0.008*"bo" + 0.007*"repeat" + 0.006*"eye" + 0.006*"ok" + 0.005*"contact" + 0.005*"husband" + 0.005*"um" + 0.005*"cos" + 0.004*"brain" + 0.004*"pregnant"'),
 (2,
  '0.007*"joke" + 0.006*"mom" + 0.005*"parents" + 0.003*"hasan" + 0.003*"clinton" + 0.003*"jokes" + 0.003*"anthony" + 0.003*"york" + 0.002*"nuts" + 0.002*"tit"'),
 (3,
  '0.006*"ahah" + 0.005*"gay" + 0.004*"son" + 0.004*"nigga" + 0.004*"gun" + 0.004*"wife" + 0.003*"trouble" + 0.003*"jesus" + 0.003*"mad" + 0.003*"sense"')]

In [27]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[(1, 'ali'),
 (2, 'anthony'),
 (3, 'bill'),
 (1, 'bo'),
 (3, 'dave'),
 (2, 'hasan'),
 (0, 'jim'),
 (0, 'joe'),
 (2, 'john'),
 (2, 'louis'),
 (0, 'mike'),
 (2, 'ricky')]

For a first pass of LDA, these kind of make sense.
* Topic 0: mom, parents [Anthony, Hasan, Louis, Ricky]
* Topic 1: husband, wife [Ali, John, Mike]
* Topic 2: guns [Bill, Bo, Jim]
* Topic 3: profanity [Dave, Joe]